In [5]:
import deviantart
import requests
import random
import sqlite3
from bs4 import BeautifulSoup
import json
import time
import requests
import requests.auth
import datetime
from requests_oauthlib import OAuth2Session
from oauthlib.oauth2 import BackendApplicationClient
import threading
import os
import pandas as pd

In [6]:
#Code to return token
def get_token():
         client = BackendApplicationClient(client_id=client_id)
         oauth = OAuth2Session(client=client)
         token = oauth.fetch_token(token_url=TOKEN_URL, client_id=client_id, client_secret=client_secret)
         # Extract the access token
         access_token = token['access_token']
         return access_token

In [7]:
#Attributes to generate tokens
client_id = "42096"
client_secret = "97080792c6d30a4178965e41f1ca15de"
TOKEN_URL = "https://www.deviantart.com/oauth2/token"
REDIRECT_URI = "https://www.deviantart.com/oauth2/authorize"

In [8]:
def get_response_rate(response):    
    if response.status_code == 200:
        return json.loads(response.content.decode('utf-8'))
    elif response.status_code == 404:
        return 'user_done'
    elif response.status_code == 429:
        return 'too_many_requests'
    elif response.status_code == 500:
        return 'server error'
    elif response.status_code == 401:
        return 'get new token'

In [9]:
#Function to get friends
def get_friends(username, page):
    token=get_token()
    api_url=f"https://www.deviantart.com/api/v1/oauth2/user/friends/{username}?access_token={token}"
    response = requests.get(api_url, params={'offset': page, 'limit': 50})
    return response

In [10]:
#Function to get watchers
def get_watchers(username, page):
    token=get_token()
    api_url=f"https://www.deviantart.com/api/v1/oauth2/user/watchers/{username}?access_token={token}"
    response = requests.get(api_url, params={'offset': page, 'limit': 50})
    return response

In [25]:
get_friends('wolfysilver',1).json()

{'has_more': True,
 'next_offset': 51,
 'results': [{'user': {'userid': '3B96BA5A-3671-5CC2-7C20-EC9A9874E006',
    'username': '22Zitty22',
    'usericon': 'https://a.deviantart.net/avatars/2/2/22zitty22.jpg?14',
    'type': 'regular'},
   'is_watching': False,
   'watches_you': False,
   'lastvisit': '2024-09-23T10:20:16-0700',
   'watch': {'friend': True,
    'deviations': True,
    'journals': False,
    'forum_threads': False,
    'critiques': False,
    'scraps': False,
    'activity': True,
    'collections': True}},
  {'user': {'userid': '65D616DD-066D-F4B5-968E-8370B74CF639',
    'username': 'acidreins',
    'usericon': 'https://a.deviantart.net/avatars/a/c/acidreins.gif',
    'type': 'regular'},
   'is_watching': False,
   'watches_you': False,
   'lastvisit': None,
   'watch': {'friend': True,
    'deviations': True,
    'journals': True,
    'forum_threads': True,
    'critiques': False,
    'scraps': False,
    'activity': True,
    'collections': True}},
  {'user': {'user

In [15]:
#Function to parse friends
def parse_friends(friends):
    users=[]
    #print(friends.keys())
    has_more=friends.get('has_more')
    #next_offset=friends['next_offset']
    for i in friends['results']:
        a={ 'username': i['user']['username'],
            'is_watching': i['is_watching'],
            'watches_you': i['watches_you'],
            'last_visit': i['lastvisit'],
            'activity': i['watch']['activity'],
            'collections': i['watch']['collections'],
            'critiques':  i['watch']['critiques'],
            'deviations': i['watch']['deviations'],
            'forum_threads': i['watch']['forum_threads'],
            'friend': i['watch']['friend'],
            'journals': i['watch']['journals'],
            'scraps': i['watch']['scraps']}
    return (has_more, a)

In [26]:
get_watchers('wolfysilver',1).json()

{'has_more': True,
 'next_offset': 51,
 'results': [{'user': {'userid': '35EC9642-6005-F301-D155-A2ED39ECF746',
    'username': 'PanchiVisuara',
    'usericon': 'https://a.deviantart.net/avatars/p/a/panchivisuara.jpg?4',
    'type': 'premium'},
   'is_watching': False,
   'lastvisit': '2024-11-28T02:18:52-0800',
   'watch': {'friend': True,
    'deviations': True,
    'journals': True,
    'forum_threads': True,
    'critiques': False,
    'scraps': False,
    'activity': True,
    'collections': True}},
  {'user': {'userid': 'E54E7980-7DCD-6915-B079-025A343E068D',
    'username': 'mermaidbanana',
    'usericon': 'https://a.deviantart.net/avatars/m/e/mermaidbanana.jpg?12',
    'type': 'premium'},
   'is_watching': False,
   'lastvisit': '2024-11-24T15:17:19-0800',
   'watch': {'friend': True,
    'deviations': True,
    'journals': True,
    'forum_threads': True,
    'critiques': False,
    'scraps': False,
    'activity': True,
    'collections': True}},
  {'user': {'userid': 'B18E08

In [27]:
def parse_watchers(watchers):
    users=[]
    #print(friends.keys())
    has_more=watchers.get('has_more')
    #next_offset=friends['next_offset']
    for i in watchers['results']:
        a={ 'username': i['user']['username'],
            'is_watching': i['is_watching'],
            'last_visit': i['lastvisit'],
            'activity': i['watch']['activity'],
            'collections': i['watch']['collections'],
            'critiques':  i['watch']['critiques'],
            'deviations': i['watch']['deviations'],
            'forum_threads': i['watch']['forum_threads'],
            'friend': i['watch']['friend'],
            'journals': i['watch']['journals'],
            'scraps': i['watch']['scraps']}
    return (has_more, a)

In [38]:
#Get all friends and watchers
def get_all_friends_and_watchers(user):
    token=get_token()
    all_friends=[]
    all_watchers=[]
    has_more=True
    for i in range(0,500):
        if has_more==True:
            resp_frnds=get_friends(user, i)
            resp_watchers = get_watchers(user,i)
            friends=get_response_rate(resp_frnds)
            watchers = get_response_rate(resp_watchers)
            #print(friends)
            if friends=='too_many_requests':
                time.sleep(60)
                friends=get_response_rate(resp_frnds)
            if friends != None:
                has_more, users=parse_friends(friends)
            else:
                return 'no friends'
            if user!= None:
                all_friends=all_friends.append(user)
            else:
                return 'no friends'

            if watchers=='too_many_requests':
                time.sleep(60)
                watchers=get_response_rate(resp_watchers)
            if watchers != None:
                has_more, users=parse_watchers(watchers)
            else:
                return 'no watchers'
            if user!= None:
                all_watchers=all_watchers.append(user)
            else:
                return 'no watchers'
    print(all_friend.info(), all_watchers.info())
    return (all_friends, all_watchers)

In [39]:
get_all_friends_and_watchers("oktiger")

AttributeError: 'NoneType' object has no attribute 'append'